In [119]:
import pandas as pd
from collections import OrderedDict


# Packages required for JSON extraction:
import json
import requests
from urllib import request
from bs4 import BeautifulSoup


# Extracting JSONs:

Here is how the sample JSON file from cleartax website looks. We should extract address from such JSONs.

In [101]:
#Json sample from Clear Tax site - written as Python String
json_sample = '''
{"taxpayerInfo": {"taxpayerInfo": {"pradr": {"ntr": "Retail Business", "addr": {"lt": "", "city": "", "bnm": "", "lg": "", "pncd": "403401", "st": "DHAVALI KAVALE", "bno": "449", "loc": "PONDA", "dst": "North Goa", "stcd": "Goa", "flno": ""}}, "stjCd": "GA007", "stj": "Ponda", "cxdt": "", "lstupdt": "21/Sep/2020", "sts": "Active", "ctb": "Proprietorship", "tradeNam": "M/S OM SAI TRADERS", "errorMsg": null, "ctjCd": "UF0303", "dty": "Regular", "ctj": "RANGE-III-PONDA", "frequencyType": null, "nba": ["Retail Business"], "gstin": "30AFRPN5938E2ZA", "adadr": [], "lgnm": "DEEPAK RAGHU NAIK", "rgdt": "27/07/2017"}, "errorResponse": null}, "filing": [], "compliance": {"filingFrequency": null}}
'''

In [102]:
#Convert Python String into JSON
data = json.loads(json_sample)

In [103]:
print(data)

{'taxpayerInfo': {'taxpayerInfo': {'pradr': {'ntr': 'Retail Business', 'addr': {'lt': '', 'city': '', 'bnm': '', 'lg': '', 'pncd': '403401', 'st': 'DHAVALI KAVALE', 'bno': '449', 'loc': 'PONDA', 'dst': 'North Goa', 'stcd': 'Goa', 'flno': ''}}, 'stjCd': 'GA007', 'stj': 'Ponda', 'cxdt': '', 'lstupdt': '21/Sep/2020', 'sts': 'Active', 'ctb': 'Proprietorship', 'tradeNam': 'M/S OM SAI TRADERS', 'errorMsg': None, 'ctjCd': 'UF0303', 'dty': 'Regular', 'ctj': 'RANGE-III-PONDA', 'frequencyType': None, 'nba': ['Retail Business'], 'gstin': '30AFRPN5938E2ZA', 'adadr': [], 'lgnm': 'DEEPAK RAGHU NAIK', 'rgdt': '27/07/2017'}, 'errorResponse': None}, 'filing': [], 'compliance': {'filingFrequency': None}}


In [104]:
print(type(data))

<class 'dict'>


In [105]:
data['taxpayerInfo']['taxpayerInfo']['tradeNam']

'M/S OM SAI TRADERS'

In [106]:
d = OrderedDict(data['taxpayerInfo']['taxpayerInfo']['pradr']['addr'])
d.move_to_end('stcd')
d.move_to_end('pncd')
d.move_to_end('bno',False)
d#.values()

OrderedDict([('bno', '449'),
             ('lt', ''),
             ('city', ''),
             ('bnm', ''),
             ('lg', ''),
             ('st', 'DHAVALI KAVALE'),
             ('loc', 'PONDA'),
             ('dst', 'North Goa'),
             ('flno', ''),
             ('stcd', 'Goa'),
             ('pncd', '403401')])

In [107]:
s = ','.join(d.values()).replace('.','')
s = [o for o in s.split(',') if len(o)]
', '.join(s)

'449, DHAVALI KAVALE, PONDA, North Goa, Goa, 403401'

In [ ]:
# Write a function that does the above extraction:
def get_address_from_json(json_data,df):
    data = json.loads(json_data)
    
    firm = data['taxpayerInfo']['taxpayerInfo']['tradeNam']
    
    d = OrderedDict(data['taxpayerInfo']['taxpayerInfo']['pradr']['addr'])
    d.move_to_end('stcd')
    d.move_to_end('pncd')
    d.move_to_end('bno',False)
    #print(d)
    s = ','.join(d.values()).replace('.','')
    s = [o for o in s.split(',') if len(o)]
    address =  ', '.join(s)
    #print(address)
    df = df.append({'Firm':firm,
                    'Address':address},ignore_index=True)
    return df

# Creating Address Dataset

In [174]:
gstn_df = pd.read_csv('Book1.csv')
gstn_df.sample(2)

C:\Users\saikr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Month,Year,st,gstin
244781,Aug,2018.0,DN,NaN
111708,Aug,2018.0,UP,19DYCPB2403M1ZL


In [181]:
gstins = gstn_df.dropna().gstin.unique()

In [182]:
len(gstins)

208676

In [176]:
address_df = pd.DataFrame(columns=['Firm','Address'])

In [179]:
#Get json 

for gstin in gstins[:4]:
    gstin = str(gstin)
    print(gstin)
    URL =  "https://cleartax.in/f/compliance-report/" + gstin + "/"
    html = request.urlopen(URL).read()
    soup = BeautifulSoup(html)
    address_df = get_address_from_json(soup.text,address_df)
    

01AAAAA0702A1Z7
01AAAAB1726H1ZI
01AAAAB3160D1ZQ
01AAAAC9093N1ZK


In [183]:
address_df.to_csv('address_sample.csv',index=False)